# Executive Summary

This notebook will attempt to answer the following research question:

    What's money got to do with it?

## PLANNING

- [X] Planning
    - [X] import libraries/packages
    - [X] configure notebook environment
    - [X] define helper functions
- [X] Acquire data
    - [X] get PEIMS financial data
    - [X] get STAAR performance data
- [X] Prepare the data
    - [X] prepare PEIMS data
        - [X] get rid of unwanted columns
        - [x] get rid of NaNs
    - [X] prepare STAAR data
        - [X] get rid of duplicates
        - [X] get rid of unwanted columns
        - [X] get rid of NaNs
        - [X] create new columns
        - [X] merge three datasets together
- [ ] Explore the data
    - [ ] examine for multicollinearity and determine truly independent variables
    - [ ] get to know the master dataset
    - [ ] feature engineering
- [ ] Modeling
    - [ ] create, fit, use XGBoost
    - [ ] gridsearch
    - [ ] retrain model using best hyperparameters
    - [ ] extract feature importances

In [1]:
# for manipulating dataframes
import pandas as pd
import numpy as np

# to print out all the outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

## ACQUIRE DATA

In [2]:
# Get the PEIMS and STAAR datasets
peims_df = pd.read_csv('../data/in/2007-2021-summaried-peims-financial-data.csv')

In [3]:
staar_df1 = pd.read_csv('../data/in/tidy_campstaar1_2012to2019.csv')
staar_df2 = pd.read_csv('../data/in/tidy_campstaar2_2013to2019.csv')

C:\Users\Dd\AppData\Local\Temp\ipykernel_13752\3902410786.py:1: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  staar_df1 = pd.read_csv('../data/in/tidy_campstaar1_2012to2019.csv')


In [4]:
ethnic_df = pd.read_csv('../data/in/Enrollment Report_Statewide_Districts_Grade_Ethnicity_2018-2019.csv', skiprows=4)

## PREPARATION

### ETHINICITY Dataset

In [5]:
ethnic_df.head()

,YEAR,REGION,COUNTY NAME,DISTRICT,DISTRICT NAME,ETHNICITY,CHARTER STATUS,GRADE,ENROLLMENT
0,2018-2019,7.0,ANDERSON COUNTY,1902.0,CAYUGA ISD,Asian,TRADITIONAL ISD/CSD,Pre-kindergarten,-999.0
1,2018-2019,7.0,ANDERSON COUNTY,1902.0,CAYUGA ISD,Black or African American,TRADITIONAL ISD/CSD,Pre-kindergarten,-999.0
2,2018-2019,7.0,ANDERSON COUNTY,1902.0,CAYUGA ISD,Two or more races,TRADITIONAL ISD/CSD,Pre-kindergarten,-999.0
3,2018-2019,7.0,ANDERSON COUNTY,1902.0,CAYUGA ISD,White,TRADITIONAL ISD/CSD,Pre-kindergarten,-999.0
4,2018-2019,7.0,ANDERSON COUNTY,1902.0,CAYUGA ISD,Black or African American,TRADITIONAL ISD/CSD,Kindergarten,2.0


In [6]:
ethnic_df = ethnic_df[['DISTRICT',
                       'ETHNICITY',
                       'ENROLLMENT',
                      ]]

In [7]:
ethnic_df.head()

,DISTRICT,ETHNICITY,ENROLLMENT
0,1902.0,Asian,-999.0
1,1902.0,Black or African American,-999.0
2,1902.0,Two or more races,-999.0
3,1902.0,White,-999.0
4,1902.0,Black or African American,2.0


In [8]:
ethnic_df['ENROLLED'] = np.where(ethnic_df['ENROLLMENT'] == -999, np.nan, ethnic_df['ENROLLMENT'])

In [9]:
ethnic_df['ENROLLED_WHITE'] = np.where(ethnic_df['ETHNICITY'] == 'White', ethnic_df['ENROLLED'], 0)

In [10]:
ethnic_df['ENROLLED_X'] = np.where(ethnic_df['ETHNICITY'] != 'White', ethnic_df['ENROLLED'], 0)

In [11]:
ethnic_df = ethnic_df.drop(columns=['ENROLLMENT', 'ENROLLED', 'ETHNICITY'])

In [12]:
# Turn dataset into something-level data
ethnic_df = ethnic_df.groupby(['DISTRICT',
                              ])[["ENROLLED_WHITE",
                                  "ENROLLED_X"]].sum().reset_index()

In [13]:
ethnic_df

,DISTRICT,ENROLLED_WHITE,ENROLLED_X
0,1902.0,447.0,76.0
1,1903.0,981.0,263.0
2,1904.0,598.0,162.0
3,1906.0,245.0,90.0
4,1907.0,907.0,2309.0
5,1908.0,764.0,682.0
6,1909.0,354.0,52.0
7,2901.0,1238.0,2959.0
8,3801.0,541.0,459.0
9,3902.0,1810.0,1090.0


In [14]:
ethnic_df['XFACTOR'] = ethnic_df["ENROLLED_X"] / (ethnic_df["ENROLLED_WHITE"] + ethnic_df["ENROLLED_X"] )

In [15]:
ethnic_df = ethnic_df.drop(columns=['ENROLLED_WHITE', 'ENROLLED_X'])

In [16]:
ethnic_df[ethnic_df['DISTRICT'] == 1902.0]

,DISTRICT,XFACTOR
0,1902.0,0.145315


In [17]:
ethnic_df = ethnic_df.rename(columns={'DISTRICT':'District',
                                      'XFACTOR':'Percentage of Non-White Students'
                        })

In [18]:
ethnic_df['District'] = ethnic_df['District'].astype('int64')

In [19]:
ethnic_df.dtypes

District                              int64
Percentage of Non-White Students    float64
dtype: object

In [20]:
# Prepend with 0's
ethnic_df['District'] = ethnic_df['District'].apply(lambda x: '{0:0>6}'.format(x))

In [21]:
ethnic_df.head()

,District,Percentage of Non-White Students
0,001902,0.145315
1,001903,0.211415
2,001904,0.213158
3,001906,0.268657
4,001907,0.717973


### STAAR Dataset

In [22]:
# Consolidate the two files into one dataframe
staar_df = pd.concat([staar_df1, staar_df2])

In [23]:
# Get rid of duplciates
staar_df = staar_df.drop_duplicates(keep='first')

In [24]:
columns_to_drop = ['data_release',
                   'data_category',
                   'data_level',
                   'release_year'
                  ]

In [25]:
# Get rid of unnecessary columns
staar_df.drop(columns=columns_to_drop, inplace=True)

In [26]:
# Prepend with 0's
staar_df['campus_number'] = staar_df['campus_number'].apply(lambda x: '{0:0>9}'.format(x))

In [27]:
# Drop 'new_rate'
staar_df.drop(columns=['new_rate'], inplace=True)

In [28]:
# Get the district number
staar_df['district'] = staar_df['campus_number'].str[:6]

In [29]:
# Turn dataset into district-level data
staar = staar_df.groupby(['test_year',
                          'district',
                          'grade_level',
                          'subject',
                          'proficiency',
                          'demog'
                         ])[["numerator", "denominator"]].sum().reset_index()

In [30]:
# Remove subsets
staar = staar[staar['grade_level'] == 'all']
staar = staar[staar['subject'] == 'all_subjects']
staar = staar[staar['demog'] == 'all_students']
staar = staar.drop(columns=['grade_level',
                            'subject',
                            'demog'
                           ])

In [31]:
staar.head()

,test_year,district,proficiency,numerator,denominator
99,2012,001902,approaches,1030,1219
110,2012,001902,masters,144,1219
120,2012,001902,meets,471,1219
372,2012,001903,approaches,1796,2279
383,2012,001903,masters,203,2279


In [32]:
staar['total_student'] = staar['denominator']

In [33]:
staar['not_passing'] = np.where(staar['proficiency'] == 'approaches', staar['numerator'], 0)
staar['passing'] = np.where(staar['proficiency'] != 'approaches', staar['numerator'], 0)

In [34]:
# Isolate 2019 test year
staar = staar[staar['test_year'] == 2019]
staar = staar.drop(columns=['test_year'])

In [35]:
staar = staar.drop(columns=['proficiency',
                            'numerator',
                            'denominator',
                            'not_passing'
                           ])

In [36]:
staar[staar['district'] == '001902']

,district,total_student,passing
2682938,001902,932,0
2682951,001902,932,269
2682964,001902,932,573


In [37]:
staar = staar.rename(columns={'district':'District',
                        'total_student':'Total Number of Students',
                        'passing':'Total Number of Passing Students'
                        })

In [38]:
# staar['District'] = staar['District'].astype('int')

In [39]:
# Turn dataset into something-level data
staar = staar.groupby(['District',
                       'Total Number of Students',
                      ])[["Total Number of Passing Students"]].sum().reset_index()

In [40]:
staar.head(10)

,District,Total Number of Students,Total Number of Passing Students
0,001902,932,842
1,001903,1991,1651
2,001904,1365,1064
3,001906,569,420
4,001907,5497,3274
5,001908,2467,1240
6,001909,635,489
7,002901,7268,4476
8,003801,1774,1493
9,003902,4693,1461


### PEIMS Dataset

In [41]:
# peims_df['DISTRICT NUMBER'] = peims_df['DISTRICT NUMBER'].str[1:]

In [44]:
peims_df.head()

,DISTRICT NUMBER,DISTRICT NAME,YEAR,GEN FUNDS-LOCAL TAX REVENUE FROM M&O,ALL FUNDS-LLOCAL TAX REVENUE FROM M&O,GEN FUNDS-STATE REVENUE,ALL FUNDS-STATE REVENUE,GEN FUNDS-FEDERAL REVENUE,ALL FUNDS-FEDERAL REVENUE,GEN FUNDS-OTHER LOCAL & INTERMEDIATE REVENUE,ALL FUNDS-OTHER LOCAL & INTERMEDIATE REVENUE,GEN FUNDS-TOTAL OPERATING REVENUE,ALL FUNDS-TOTAL OPERATING REVENUE,GEN FUNDS-LOCAL PROPERTY TAXES FROM I&S,ALL FUNDS-LOCAL PROPERTY TAXES FROM I&S,GEN FUNDS-STATE DEBT FUNDS,ALL FUNDS-STATE DEBT FUNDS,GEN FUNDS-OTHER RECEIPTS,ALL FUNDS-OTHER RECEIPTS,GEN FUNDS-OTHER REVENUE,ALL FUNDS-OTHER REVENUE,GEN FUNDS-TOTAL OPERATING REVENUE AND OTHER REVENUE,ALL FUNDS-TOTAL OPERATING REVENUE AND OTHER REVENUE,GEN FUNDS-EQUITY TRANSFERS,ALL FUNDS-EQUITY TRANSFERS,GEN FUNDS-TOT DEBT SERV FIN AND TRS EST REV,ALL FUNDS-TOT DEBT SERV FIN AND TRS EST REV,GEN FUNDS-TOTAL OPERATING REVENUE AND OTHER REVENUE AND RECPATUR,ALL FUNDS-TOTAL OPERATING REVENUE AND OTHER REVENUE AND RECPATUR,GEN FUNDS-DEBT SERVICE FINANCING RELATED REVENUE,ALL FUNDS-DEBT SERVICE FINANCING RELATED REVENUE,GEN FUNDS-ESTIMATED STATE TRS CONTRINUTIONS,ALL FUNDS-ESTIMATED STATE TRS CONTRINUTIONS,GEN FUNDS-TOTAL DEBT SERVICE FINANCING AND TRS ESTIMATE REVENUE,ALL FUNDS-TOTAL DEBT SERVICE FINANCING AND TRS ESTIMATE REVENUE,"GEN FUNDS-TOTAL OPERATING, OTR, DEBT SERV FIN, AND TRS EST REVEN","ALL FUNDS-TOTAL OPERATING, OTR, DEBT SERV FIN, AND TRS EST REVEN",GEN FUNDS-TOTAL PAYROLL EXPENDITURES,ALL FUNDS-TOTAL PAYROLL EXPENDITURES,GEN FUNDS-TOTAL PROFESSIONAL & CONTRACTED SERVICES EXPENDITURES,ALL FUNDS-TOTAL PROFESSIONAL & CONTRACTED SERVICES EXPENDITURES,GEN FUNDS-TOTAL SUPPLIES & MATERIALS EXPENDITURES,ALL FUNDS-TOTAL SUPPLIES & MATERIALS EXPENDITURES,GEN FUNDS-TOTAL OTHER OPERATING EXPENDITURES,ALL FUNDS-TOTAL OTHER OPERATING EXPENDITURES,GEN FUNDS-TOTAL OPERATING EXPENDITURES BY OBJ,ALL FUNDS-TOTAL OPERATING EXPENDITURES BY OBJ,GEN FUNDS-TOTAL DEBT SERVICE EXPEND BY OBJ,ALL FUNDS-TOTAL DEBT SERVICE EXPEND BY OBJ,GEN FUNDS-TOTAL CAPITAL OUTLAY EXPEND BY OBJ,ALL FUNDS-TOTAL CAPITAL OUTLAY EXPEND BY OBJ,GEN FUNDS-TOTAL NON-OPER EXPENDITURES BY OBJ,ALL FUNDS-TOTAL NON-OPER EXPENDITURES BY OBJ,GEN FUNDS-TOTAL NON-OPER AND OPER OEXPENDITURES BY OBJ,ALL FUNDS-TOTAL NON-OPER AND OPER OEXPENDITURES BY OBJ,"GEN FUNDS-INSTRUCTION + TRANSFER EXPEND-FCT11,95","ALL FUNDS-INSTRUCTION + TRANSFER EXPEND-FCT11,95","GEN FUNDS-INSTRUC RESOURCE MEDIA SERVICE EXP, FCT12","ALL FUNDS-INSTRUC RESOURCE MEDIA SERVICE, FCT12","GEN FUNDS-CURRICULUM/STAFF DEVELOPMENT EXP, FCT13","ALL FUNDS-CURRICULUM/STAFF DEVELOPMENT EXP, FCT13","GEN FUNDS-INSTRUC LEADERSHIP EXPEND, FCT21","ALL FUNDS-INSTRUC LEADERSHIP EXPEND, FCT21","GEN FUNDS-CAMPUS ADMINISTRATION EXPEND, FCT23","ALL FUNDS-CAMPUS ADMINISTRATION EXPEND, FCT23","GEN FUNDS-GUIDANCE & COUNSELING SERVICES EXP, FCT31","ALL FUNDS-GUIDANCE 7 COUNSELING SERVICES EXP, FCT31","GEN FUNDS-SOCIAL WORK SERVICES EXP, FCT32","ALL FUNDS-SOCIAL WORK SERVICES EXP, FCT32","GEN FUNDS-HEALTH SERVICES EXP, FCT33","ALL FUNDS-HEALTH SERVICES EXP, FCT33","GEN FUNDS-TRANSPORTATION EXPENDITURES, FCT34","ALL FUNDS-TRANSPORTATION EXPENDITURES, FCT34","GEN FUNDS-FOOD SERVICE EXPENDITURES, FCT35","ALL FUNDS-FOOD SERVICE EXPENDITURES, FCT35","GEN FUNDS-EXTRACURRICULAR EXPENDITURES, FCT36","ALL FUNDS-EXTRACURRICULAR EXPENDITURES,FCT36","GEN FUNDS-GENERAL ADMINISTRAT EXPEND-FCT41,80,92","ALL FUNDS-GENERAL ADMINISTRAT EXPEND-FCT41,80,92","GEN FUNDS-PLANT MAINTENANCE/OPERA EXPEND, FCT51","ALL FUNDS-PLANT MAINTENANCE/OPERA EXPEND, FCT51","GEN FUNDS-SECURITY/MONITORING SERVICE EXPEND, FCT52","ALL FUNDS-SECURITY/MONITORING SERVICE EXPEND, FCT5","GEN FUNDS-DATA PROCESSING SERVICES EXPEND, FCT53","ALL FUNDS-DATA PROCESSING SERVICES, FCT53","GEN FUNDS-COMMUNITY SERVICES, FCT61","ALL FUNDS-COMMUNITY SERVICES, FCT61",GEN FUNDS-TOTAL OPERATE EXPEND BY FUNCTION,ALL FUNDS-TOTAL OPERATE EXPEND BY FUNCTION,GEN FUNDS-NON-OPER EXP BY FUNCTION(1X-9X)(65XX),ALL FUNDS-NON-OPER EXP BY FUNCTION(1X-9X)(65XX),GEN 

In [45]:
peims = peims_df.drop(columns=['DISTRICT NAME'])

In [46]:
# Remove all columns between column name 'B' to 'D'
peims = peims.drop(peims.loc[:, 'GEN FUNDS-LOCAL TAX REVENUE FROM M&O':'ALL FUNDS-TOTAL OPERATING, OTR, DEBT SERV FIN, AND TRS EST REVEN'].columns, axis=1)

In [47]:
# Isolate 2019 test year
peims = peims[peims['YEAR'] == 2019]
peims = peims.drop(columns=['YEAR'])

In [48]:
# Remove columns that starts with 'GEN'
columns_to_keep = [c for c in peims.columns if c.lower()[:3] != 'gen']
peims = peims[columns_to_keep]

In [49]:
# Remove Prefix 'all funds'
peims.columns = peims.columns.str.replace("ALL FUNDS-", "")

In [50]:
# Padd District numbers with 0's
peims['DISTRICT NUMBER'] = peims['DISTRICT NUMBER'].str.zfill(6)

In [51]:
peims = peims.drop(peims.loc[:, 'TOTAL OPERATING EXPENDITURES BY OBJ':'TOTAL NON-OPER AND OPER OEXPENDITURES BY OBJ'].columns, axis=1)

In [52]:
peims = peims.drop(peims.loc[:, 'TOTAL OPERATE EXPEND BY FUNCTION':'TOT OPER AND NON-OPER EXP BY FUNCTION'].columns, axis=1)

In [53]:
peims = peims.drop(peims.loc[:, 'NON OPER EXP BY PIC(65XX)':'EINTRAN4'].columns, axis=1)

In [54]:
peims = peims.drop(peims.loc[:, 'INTERGOVERN CHARGES EXPEND':'FALL SURVEY ENROLLMENT'].columns, axis=1)

In [55]:
peims = peims.drop(peims.loc[:, 'PREKINDERGARTEN EXPEND BILINGUAL--32':'PREKINDERGARTEN EXPEND SPECIAL ED--32'].columns, axis=1)

In [56]:
peims = peims.rename(columns={'DISTRICT NUMBER':'DISTRICT'})

In [57]:
# Remove word 'EXPENDITURES'
peims.columns = peims.columns.str.replace("EXPENDITURES", "")

In [58]:
# Remove word 'EXPEND'
peims.columns = peims.columns.str.replace("EXPEND", "")

In [59]:
# Remove word 'EXP'
peims.columns = peims.columns.str.replace("EXP", "")

In [60]:
# Remove word 'TOTAL'
peims.columns = peims.columns.str.replace("TOTAL", "")

In [61]:
# Remove word 'FCT'
peims.columns = peims.columns.str.replace("FCT", "")

In [62]:
# Remove dougble dashes
peims.columns = peims.columns.str.replace("--", "-")

In [63]:
# Remove double space opposite comma
peims.columns = peims.columns.str.replace(" , ", ",")

In [64]:
# Remove leading and treiling spaces
peims.columns = peims.columns.str.strip()

In [65]:
peims = peims.rename(columns={'DISTRICT':'District',
                              'PAYROLL':'Payroll',
                              'PROFESSIONAL & CONTRACTED SERVICES':'Professional & Contracted',
                              'SUPPLIES & MATERIALS':'Supplies & Materials',
                              'OTHER OPERATING':'Other Operating',
                              'INSTRUCTION + TRANSFER -11,95':'Instruction & Transfer',
                              'INSTRUC RESOURCE MEDIA SERVICE, 12':'Instructional Resource Media',
                              'CURRICULUM/STAFF DEVELOPMENT,13':'Curriculum/Staff Development',
                              'INSTRUC LEADERSHIP,21':'Instructional Leadership',
                              'CAMPUS ADMINISTRATION,23':'Campus Administration',
                              'GUIDANCE 7 COUNSELING SERVICES,31':'Guidance & Counseling',
                              'SOCIAL WORK SERVICES,32':'Social Work',
                              'HEALTH SERVICES,33':'Health Services',
                              'TRANSPORTATION,34':'Transportation',
                              'FOOD SERVICE,35':'Food Service',
                              'EXTRACURRICULAR ,36':'Extracurricular',
                              'GENERAL ADMINISTRAT -41,80,92':'General Administration',
                              'PLANT MAINTENANCE/OPERA,51':'Plant Maintenance/Operation',
                              'SECURITY/MONITORING SERVICE,5':'Security & Monitoring',
                              'DATA PROCESSING SERVICES, 53':'Data Processing',
                              'COMMUNITY SERVICES, 61':'Community Services',
                              'REGULAR PROGRAM -11':'Regular Program',
                              'GIFTED/TALENTED PROGRAM -21':'Gifted & Talented Program',
                              'CAREER & TECHNOLOGY PGM -22':'Career & Technology Program',
                              'STUDENTS WITH DISABILITIES PGM -23':'Students with Disabilities',
                              'STATE COMPENSATORY ED -24, 29, 30, 34':'State Compensatory Education',
                              'BILINGUAL PROGRAM -25':'Bilingual Program',
                              'HIGH SCHOOL ALLOTMENT PROGRAM-91':'High School Allotment',
                              'PREKINDERGARTEN-32,35':'Pre-K',
                              'ATHLETICS PROGRAM-91':'Athletics Program',
                              'UNDISTRIBUTED PROGRAM -99':'Undistributed Program',
                              'PROGRAM OPERATING':'Total Program Expenditures',
                              'OTHER USES':'Other Uses'
                             })

In [66]:
peims['District'] = peims['District'].str[1:]

In [67]:
peims.head()

,District,Payroll,Professional & Contracted,Supplies & Materials,Other Operating,Instruction & Transfer,Instructional Resource Media,Curriculum/Staff Development,Instructional Leadership,Campus Administration,Guidance & Counseling,Social Work,Health Services,Transportation,Food Service,Extracurricular,General Administration,Plant Maintenance/Operation,Security & Monitoring,Data Processing,Community Services,Regular Program,Gifted & Talented Program,Career & Technology Program,Students with Disabilities,State Compensatory Education,Bilingual Program,High School Allotment,Pre-K,Athletics Program,Undistributed Program,Total Program Expenditures,Other Uses
12,001902,6025217,1075904,648206,809559,4649118,66490,4986,270353,306385,998314,0,37882,293070,287406,413755,284553,773085,0,173489,0,2778638,3968,251350,3005575,273747,9599,40285,32890,304174,1858660,8558886,48633
27,001903,9093950,1514689,784631,303052,7043892,117860,33175,66374,574699,202086,0,33657,422887,630202,598484,558948,1248908,13530,151120,500,5313722,93,852319,1028587,799037,0,101243,0,339045,3262276,11696322,102465
42,001904,6659596,927209,937810,278109,4611747,51126,157830,0,466345,199338,0,102385,38800,411195,754465,539512,1014501,45482,409998,0,3945494,10154,552217,726827,377013,0,59567,114404,571388,2445660,8802724,481
57,001906,3134475,373513,408024,105878,2087166,19990,0,7905,379101,75235,0,40628,148301,257465,210240,201520,465549,10415,118375,0,1499301,14498,164641,447072,402415,2706,48748,29920,0,1412589,4021890,53786
72,001907,25587063,5603896,4134969,1048416,18807861,167823,535649,1033275,2201907,1443630,170074,208736,1442619,2071781,1422648,1287489,3937087,242658,1006175,394932,15527277,39671,1625090,2422707,3147717,231026,302531,923035,1214433,10940857,36374344,0


In [68]:
staar.shape
peims.shape
ethnic_df.shape

(1251, 3)

(1200, 33)

(1201, 2)

In [69]:
ethnic_df.to_csv('../data/inter/clean_ethnic_2019.csv', index=False)

In [70]:
staar.to_csv('../data/inter/clean_staar_2019.csv', index=False)

In [71]:
peims.to_csv('../data/inter/clean_peims_2019.csv', index=False)